# 08wk-1: Pandas (1)

최규빈  
2023-04-24

<a href="https://colab.research.google.com/github/guebin/PP2023/blob/main/posts/02_DataScience/2023-04-24-8wk-1.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 강의영상

> youtube:
> <https://youtube.com/playlist?list=PLQqh36zP38-xqAT5XH-YhYj1s2WQWhKE8>

# import

In [6]:
import numpy as np
import pandas as pd

# pandas 개발동기

## 부분 데이터 꺼내기: 판다스를 왜 써야할까?

`-` 예시1: 때로는 인덱스로 때로는 key로 데이터를 부르고 싶다.

In [7]:
np.random.seed(43052)
att = np.random.choice(np.arange(10,21)*5,20)
rep = np.random.choice(np.arange(5,21)*5,20)
mid = np.random.choice(np.arange(0,21)*5,20)
fin = np.random.choice(np.arange(0,21)*5,20)
key = ['2022-12'+str(s) for s in np.random.choice(np.arange(300,501),20,replace=False)]

학번 ’2022-12363’에 해당하는 학생의 출석점수를 알고 싶다면?

(풀이1) – dct로 자료를 저장하고 출력

In [8]:
dct = {'att':{key[i]:att[i] for i in range(20)}, 
       'rep':{key[i]:rep[i] for i in range(20)}, 
       'mid':{key[i]:mid[i] for i in range(20)}, 
       'fin':{key[i]:fin[i] for i in range(20)}}
#dct

In [9]:
dct['att']['2022-12363']

(풀이2) – ndarray로 자료를 저장하고 출력

In [16]:
arr = np.array([att,rep,mid,fin,key]).T
arr

In [91]:
arr[arr[:,-1] == '2022-12363',0] # 읽기어려운 코드

**(풀이2)가 (풀이1)에 비하여 불편한 점**

-   arr 마지마칼럼이 student id 이고 첫번째 칼럼은 att라는 사실을
    암기하고 있어야 한다.
-   자료형이 문자로 강제로 바뀌어서 저장되어있음
-   작성한 코드의 가독성이 없다. (위치로 접근하기 때문)

`-` 요약: hash 스타일로 정보를 추출하는 것이 유용할 때가 있다. 그리고
보통 hash 스타일로 정보를 뽑는 것이 유리하다. (사실 numpy는 정보추출을
위해 개발된 자료형이 아니라 행렬 및 벡터의 수학연산을 지원하기 위해
개발된 자료형이다)

`-` 소망: 정보를 추출할때는 hash 스타일도 유용하다는 것은 이해함 $\to$
하지만 나는 가끔 넘파이스타일로 정보를 뽑고 싶은걸? 그리고 딕셔너리
형태가 아니고 엑셀처럼(행렬처럼) 데이터를 보고 싶은걸? $\to$ pandas의
개발

## 엑셀처럼 데이터를 테이블 형태로 정리하고 싶다

(방법1) – 넘파이

In [19]:
arr

(방법2) – 판다스 with stacked dict

In [20]:
df = pd.DataFrame(dct)
df.head()

(방법3) – 판다스 with index

In [21]:
df = pd.DataFrame({'att':att,'rep':rep,'mid':mid,'fin':fin},index=key)
df.head()

## 해싱으로 원하는 정보를 뽑으면 좋겠다 (마치 딕셔너리처럼)

`-` 예제1: 출석점수를 출력 (딕셔너리가 되면 판다스도 된다)

In [22]:
# dct['att']
df['att']

`-` 예제2: 학번 `2022-12380` 의 출석점수 출력

In [23]:
#dct['att']['2022-12380']
df['att']['2022-12380']

## 인덱싱으로 정보를 뽑는 기능도 지원을 하면 좋겠다 (마치 리스트나 넘파이처럼)

`-` 예제1: 첫번째 학생의 기말고사 성적을 출력하고 싶다.

In [24]:
df.iloc[0,-1]

-   벼락치기: df에서 iloc이라는 특수기능을 이용하면 넘파이 인덱싱처럼
    원소출력이 가능하다.

> df는 딕셔너리 같은것이지만 df.iloc은 넘파이같은것이라고 생각하면 된다.

`-` 예제2: 홀수번째 학생 의 점수를 뽑고 싶다. (홀수번째 학생은 인덱스
0,2,4,… 에 대응)

In [25]:
df.iloc[::2,:]

`-` 예제3: 맨 끝에서 3명의 점수를 출력하고 싶다.

In [26]:
df.iloc[-3:,:]

`-` 예제4: 맨 끝에서 3명의 점수중 마지막 2개의 칼럼만 출력하고 싶다.

In [27]:
df.iloc[-3:,-2:]

## 궁극: 해싱과 인덱싱을 모두 지원하는 아주 우수한 자료형을 만들고 싶음

`-` 예제1: ’mid \>= 20 and att \<60’인 학생들의 ’fin’을 출력

(방법1) query

-   데이터베이스 스타일

In [28]:
df.query('mid>=20 and att<60')['fin']

(방법2) numpy

In [29]:
arr[(arr[:,2].astype(dtype=np.int64) >= 20) & (arr[:,0].astype(dtype=np.int64) < 60),3]

`-` 예제2: ’중간고사점수\<기말고사점수’인 학생들의 출석점수 평균을
구하자.

In [30]:
df.query('mid<fin')['att'].mean()

# pandas 공부 1단계

## 데이터프레임 선언

`-` 방법1: dictionary에서 만든다.

In [31]:
pd.DataFrame({'att':[30,40,50],'mid':[50,60,70]})

In [32]:
pd.DataFrame({'att':(30,40,50),'mid':(50,60,70)})

In [33]:
pd.DataFrame({'att':np.array([30,40,50]),'mid':np.array([50,60,70])})

`-` 방법: 2차원 ndarray에서 만든다.

In [34]:
np.arange(2*3).reshape(2,3)

In [35]:
pd.DataFrame(np.arange(2*3).reshape(2,3))

## 열의 이름 부여

`-` 방법1: 딕셔너리를 통하여 만들면 딕셔너리의 key가 자동으로 열의
이름이 된다.

In [36]:
pd.DataFrame({'att':np.array([30,40,50]),'mid':np.array([50,60,70])})

`-` 방법2: pd.DataFrame()의 옵션에 columns를 이용

In [37]:
pd.DataFrame(np.arange(2*3).reshape(2,3),columns=['X1','X2','X3'])

`-` 방법3: df.columns에 원하는 열이름을 덮어씀 (1)

In [38]:
df=pd.DataFrame(np.arange(2*3).reshape(2,3))
df

In [41]:
df.columns = ['X1','X2','X3']
df

`-` 방법4: df.columns에 원하는 열이름을 덮어씀 (2)

In [42]:
df=pd.DataFrame(np.arange(2*3).reshape(2,3))
df

In [43]:
df.columns = pd.Index(['X1','X2','X3'])

In [44]:
df

방법4가 방법3의 방식보다 컴퓨터가 이해하기 좋다. (= 불필요한 에러 혹은
경고메시지를 방지할 수 있다)

## 행의 이름 부여

`-` 방법1: 중첩 dict이면 nested dic의 key가 알아서 행의 이름으로 된다.

In [45]:
pd.DataFrame({'att':{'guebin':30, 'iu':40, 'hynn':50} , 'mid':{'guebin':5, 'iu':45, 'hynn':90}})

`-` 방법2: pd.DataFrame()의 index 옵션 이용

In [46]:
pd.DataFrame({'att':[30,40,50] , 'mid':[5,45,90]},index=['guebin','iu','hynn'])

`-` 방법3: df.index에 덮어씌움

In [47]:
df=pd.DataFrame({'att':[30,40,50] , 'mid':[5,45,90]})
df

In [48]:
df.index = pd.Index(['guebin','iu','hynn'])
#df.index = ['guebin','iu','hynn'] <- 이것도 실행가능하기는함 
df

`-` 방법4: df.set_index() 를 이용하여 덮어씌운다

In [49]:
df=pd.DataFrame({'att':[30,40,50] , 'mid':[5,45,90]})
df

In [50]:
df.set_index(pd.Index(['guebin','iu','hynn']))

In [51]:
df.set_index(['guebin','iu','hynn'])

In [52]:
df.set_index([['guebin','iu','hynn']]) # 꺽쇠를 한번 더 넣어주면 에러를 피할수 있다. 

-   그러나 이런 코드를 권장하지 않음

## 자료형, len, shape, for문의 반복변수

In [53]:
df = pd.DataFrame({'att':[30,40,50],'mid':[5,45,90]})
df

`-` type

In [54]:
type(df)

`-` len

In [55]:
len(df) # row의 갯수 

`-` shape

In [56]:
df.shape 

`-` for문의 반복변수

In [57]:
for k in df:
    print(k) # 딕셔너리같죠

att
mid

In [58]:
for k in {'att':[30,40,50],'mid':[5,45,90]}: 
    print(k)

att
mid

**참고:** df는 진짜 딕셔너리 느낌 강해요

In [59]:
df.keys()

In [62]:
for k,v in df.items():
    print(k)

att
mid

## pd.Series

`-` 2차원 ndarray가 pd.DataFrame에 대응한다면 1차원 ndarray는
pd.Series에 대응한다.

In [63]:
a=pd.Series(np.random.randn(10))
a

In [64]:
type(a)

In [65]:
len(a)

In [66]:
a.shape

In [67]:
for value in a: 
    print(value)

0.10617283591748639
0.7237590624253404
0.21798967912700873
0.1940223087322443
-0.6889899757985083
-0.3516696436204985
0.9909329773184973
1.2121468150185186
-0.6089654373693767
0.03254898346416765

# pandas 공부 2단계

`-` 데이터

In [68]:
np.random.seed(43052)
att = np.random.choice(np.arange(10,21)*5,20)
rep = np.random.choice(np.arange(5,21)*5,20)
mid = np.random.choice(np.arange(0,21)*5,20)
fin = np.random.choice(np.arange(0,21)*5,20)
key = ['2022-12'+str(s) for s in np.random.choice(np.arange(300,501),20,replace=False)]

In [69]:
df = pd.DataFrame({'att':att,'rep':rep,'mid':mid,'fin':fin},index=key)
df.head()

## 열의 선택

`-` 방법1: `df[]` + 칼럼이름, 칼럼이름의 list

In [87]:
# df['att'] # 칼럼이름 
# df[['att']] # 칼럼이름의 list 
# df[['att','rep']] # 칼럼이름의 list 

`-` 방법2: `df.iloc[:,]` + 정수, 정수의 list, range, 슬라이싱,
스트라이딩, bool의 list

In [88]:
# df.iloc[:,0] # 정수
# df.iloc[:,[0]] # 정수의 list 
# df.iloc[:,[0,1]] # 정수의 list 
# df.iloc[:,range(2)] # range
# df.iloc[:,-2:] # 슬라이싱
# df.iloc[:,1::2] # 스트라이딩
# df.iloc[:,[True,True,False,False]] # bool의 list 

`-` 방법3: `df.loc[:,]` + 칼럼이름, 컬럼이름의 list, 칼럼이름으로
슬라이싱($\star$), 칼럼이름으로 스트라이딩($\star$), bool의 list

In [89]:
# df.loc[:,'att'] # 칼럼이름
# df.loc[:,['att']] # 칼럼이름의 list 
# df.loc[:,['att','rep']] # 칼럼이름의 list 
# df.loc[:,'rep':'mid'] # 칼럼이름으로 슬라이싱 
# df.loc[:,'rep'::2] # 칼럼이름으로 스트라이딩
# df.loc[:,[True,False,False,True]] # bool의 list

## 행의 선택

**여기서는 `df=중첩된list` 라고 생각해야 코드가 잘 읽힌다.**

`-` 방법1: `df.iloc[]` + 정수, 정수의리스트, range, 슬라이싱,
스트라이딩, bool의 list

In [75]:
# df.iloc[0] # 정수 
# df.iloc[[0]] # 정수의 list 
# df.iloc[[0,1]] # 정수의 list 
# df.iloc[range(2)] # range
# df.iloc[-2:] # 슬라이싱
# df.iloc[1::2] # 스트라이딩
# df.iloc[[True]+[False]*19] # bool의 list 
# df.iloc[list(df['att']>70)] # bool의 list 

**여기서는 `df=2차원array`라고 생각해야 코드가 잘 읽힌다.**

`-` 방법1: `df.iloc[,:]` + 정수, 정수의리스트, range, 슬라이싱,
스트라이딩, bool의 list

In [76]:
# df.iloc[0,:] # 정수 
# df.iloc[[0],:] # 정수의 list 
# df.iloc[[0,1],:] # 정수의 list 
# df.iloc[range(2),:] # range
# df.iloc[-2:,:] # 슬라이싱
# df.iloc[1::2,:] # 스트라이딩
# df.iloc[[True]+[False]*19,:] # bool의 list 
# df.iloc[list(df['att']>70),:] # bool의 list 

`-` 방법2: `df.loc[,:]` + 인덱스이름의 리스트, 인덱스이름으로
슬라이싱($\star$), 인덱스이름으로 스트라이딩($\star$), bool의 list

In [77]:
# df.loc['2022-12380',:] # 인덱스이름 
# df.loc[['2022-12380','2022-12370'],:] # 인덱스이름의 리스트
# df.loc['2022-12452':,:] # 인덱스이름으로 슬라이싱
# df.loc['2022-12380'::3,:] # 인덱스이름으로 스트라이딩
# df.loc[list(df['att']>70),:] # bool의 list 
# df.loc[df['att']>70,:] # bool의 list 

## 안썼으면 좋겠는 코드

`-` 제가 안쓰는 코드1:

In [78]:
df['2022-12380':'2022-12370']

이러면 내 입장에서는 마치 아래가 동작할 것 같잖아..

In [79]:
df['2022-12380']

`-` 제가 안쓰는 코드2: bool의 list를 사용할때 iloc은 가급적 쓰지마세요

In [83]:
df.iloc[list(df['att']<80),:]

이러면 마치 아래도 동작할 것 같잖아..

In [84]:
df.iloc[df['att']<80,:]

`-` 참고: 맨날 틀리는 코드

In [86]:
df.loc['att']

# HW

`(1)` 아래와 같은 데이터 프레임을 선언하라.

In [14]:
from IPython.core.display import HTML 
HTML('<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>A</th>\n      <th>B</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>1</td>\n      <td>-2</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>2</td>\n      <td>-3</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>3</td>\n      <td>-4</td>\n    </tr>\n  </tbody>\n</table>')

(풀이)

In [54]:
df = pd.DataFrame({'A':[1,2,3], 'B':[-2,-3,-4]})
df

`(2)` Column을 이름을 X1, X2로 변경하라. 출력결과는 아래와 같아야 한다.

In [16]:
from IPython.core.display import HTML 
HTML('<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>X1</th>\n      <th>X2</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>1</td>\n      <td>-2</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>2</td>\n      <td>-3</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>3</td>\n      <td>-4</td>\n    </tr>\n  </tbody>\n</table>')

(풀이)

In [55]:
df.columns = pd.Index(['X1','X2'])
df

`(3)-(5)` 아래와 같은 자료를 고려하자.

In [56]:
df = pd.DataFrame(np.random.normal(size=(100,5)),columns=list('ABCDE'))
df

`(3)` B,D열을 선택하라.

(풀이)

In [57]:
df.loc[:,['B','D']]

`(4)` 마지막 10개의 row를 출력하라.

(풀이)

In [58]:
df.iloc[-10:]

`(5)` A,B 열의 처음 10개의 row를 출력하라.

(풀이)

In [60]:
df.loc[:,['A','B']].iloc[:10]

`(6)-(9)` 아래와 같은 자료를 고려하자.

In [63]:
df=pd.read_csv('https://raw.githubusercontent.com/PacktPublishing/Pandas-Cookbook/master/data/movie.csv')
df

`(6)` 이 데이터프레임에는 몇개의 컬럼이 있는지 count하라.

**hint**: df.columns의 len을 조사

(풀이)

In [64]:
len(df.columns)

`(7)` 데이터프레임의 컬럼이름이 c혹은 d로 시작하는 열은 몇개 있는지
세어보라.

**hint:** 아래의 코드를 관찰

In [28]:
lst = ['color', 'director_name', 'num_critic_for_reviews', 'duration'] 
[l for l in lst if l[0]=='c' or l[0]=='d']

(풀이)

In [67]:
[l for l in df.columns if l[0]=='c' or l[0]=='d']

In [68]:
len([l for l in df.columns if l[0]=='c' or l[0]=='d'])

`(8)` 이 데이터프레임에서 ’actor’라는 단어가 포함된 열이 몇개있는지
세어보라.

(풀이)

In [70]:
[l for l in df.columns if 'actor' in l]

In [72]:
len([l for l in df.columns if 'actor' in l])

`(9)` 이 데이터프레임에서 ’actor’라는 단어가 포함된 열을 출력하라.

**hint**: 아래의 코드를 관찰하라.

In [39]:
_df = pd.DataFrame(
    np.random.randint(1,200,size=(100,2)),
    columns=['director_facebook_likes', 'actor_3_facebook_likes']
)
_df

In [40]:
_df.loc[:,['actor' in colname for colname in _df.columns]]

(풀이)

In [81]:
df.loc[:, ['actor' in l for l in df.columns]]